In [6]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof

In [7]:
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing


In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
minerals = ["Q", "P", "K", "B", "O", "A"]

modal_mineralogy = np.array([[0.16331256, 0.46387757, 0.12683457, 0.05758711, 0.01078412, 0.17760407],   # AZ
                             [0.27453254, 0.40124204, 0.24993859, 0.02509231, 0.00454988, 0.04464465],   # CA-EW
                             [0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603],   # CA-NS
                             [0.19781274, 0.41766034, 0.16121575, 0.06478527, 0.00894429, 0.14958161],   # GR
                             [0.25983144, 0.46392506, 0.15827298, 0.04370113, 0.00689158, 0.06737782],   # MT
                             [0.22419158, 0.48490082, 0.05641885, 0.11571367, 0.00000001, 0.11877509]])

csds_AZ_means = np.array([0.120, 0.174, 0.112, 0.155, 0.066, 0.079])
csds_AZ_stds = np.array([0.359, 0.356, 0.455, 0.399, 0.328, 0.452])

csds_CA_EW_means = np.array([0.247, 0.333, 0.292, 0.212, 0.090, 0.127])
csds_CA_EW_stds = np.array([0.986, 0.640, 0.497, 0.889, 0.275, 0.502])

csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

csds_GR_means = np.array([0.244, 0.269, 0.222, 0.251, 0.059, 0.107])
csds_GR_stds = np.array([0.312, 0.450, 0.444, 0.262, 0.378, 0.313])

csds_MT_means = np.array([0.220, 0.227, 0.160, 0.153, 0.078, 0.075])
csds_MT_stds = np.array([0.472, 0.415, 0.344, 0.363, 0.315, 0.599])

csds_WA_means = np.array([0.342, 0.385, 0.241, 0.481, 0.001, 0.130])
csds_WA_stds = np.array([0.542, 0.478, 0.386, 0.467, 0.001, 0.494])




[0.19781274 0.41766034 0.16121575 0.06478527 0.00894429 0.14958161]


In [15]:
SedGen_CA_NS = SedGen(minerals, 1e8, modal_mineralogy[2], csds_CA_NS_means, csds_CA_NS_stds)

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|
Counting interfaces...
Correcting interface arrays for consistency...
too much Q 2
too few P -1
too few K -1
all good B 0
all good O 0
all good A 0
Initializing crystal size array... |Q|P|K|B|O|A|
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...
Initializing discretization for model's weathering...
100/100
---SedGen model initialization finished succesfully---


In [16]:
SedGen_CA_NS_mech = SedGen_CA_NS.weathering(operations = ["intra_cb","inter_cb"], timesteps = 100, inplace = False)

After 67 steps all pcgs have been broken down to mcg


In [18]:
SedGen_CA_NS_chem = SedGen_CA_NS.weathering(operations = ["chem_mcg", "chem_pcg"], timesteps = 100, inplace = False)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function bincount at 0x00000263B063BA60>) found for signature:
 
 >>> bincount(array(uint8, 1d, C), weights=array(float64, 1d, C), minlength=int64)
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'np_bincount': File: numba\np\arraymath.py: Line 3445.
    With argument(s): '(array(uint8, 1d, C), weights=array(float64, 1d, C), minlength=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypeError: np_bincount() got an unexpected keyword argument 'minlength'[0m
  raised from C:\Users\quint\anaconda3\envs\sedgen\lib\site-packages\numba\core\typing\templates.py:675
[0m
[0m[1mDuring: resolving callee type: Function(<function bincount at 0x00000263B063BA60>)[0m
[0m[1mDuring: typing of call at d:\unif\masterthesis\jupyter\sedgen\src\sedgen\general.py (139)
[0m
[1m
File "..\..\s

In [19]:
SedGen_CA_NS_both = SedGen_CA_NS.weathering(operations = ["intra_cb", "inter_cb", "chem_mcg", "chem_pcg"], timesteps = 100, inplace = False)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function bincount at 0x00000263B063BA60>) found for signature:
 
 >>> bincount(array(uint8, 1d, C), weights=array(float64, 1d, C), minlength=int64)
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'np_bincount': File: numba\np\arraymath.py: Line 3445.
    With argument(s): '(array(uint8, 1d, C), weights=array(float64, 1d, C), minlength=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypeError: np_bincount() got an unexpected keyword argument 'minlength'[0m
  raised from C:\Users\quint\anaconda3\envs\sedgen\lib\site-packages\numba\core\typing\templates.py:675
[0m
[0m[1mDuring: resolving callee type: Function(<function bincount at 0x00000263B063BA60>)[0m
[0m[1mDuring: typing of call at d:\unif\masterthesis\jupyter\sedgen\src\sedgen\general.py (139)
[0m
[1m
File "..\..\s